## 1. Quá trình dự báo trên mô hình GRU

Thêm thư viện

In [7]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import GRUV2 as GRU

Nạp dữ liệu từ file csv

In [8]:

start = '01/06/2020'
end = '01/06/2022'
pre_day = 30
stock="ethereum"
# df = investpy.get_crypto_historical_data(crypto=stock ,from_date=start, to_date=end)
df=pd.read_csv('ethereum_train_8_2.csv')
df = pd.DataFrame(df)
df

,Date,Open,High,Low,Close,Volume,Currency
0,2020-06-01,0.23180,0.24981,0.23085,0.24797,11970598,USD
1,2020-06-02,0.24801,0.25321,0.22795,0.23794,12686011,USD
2,2020-06-03,0.23790,0.24476,0.23384,0.24452,8151002,USD
3,2020-06-04,0.24453,0.24641,0.23668,0.24337,8749785,USD
4,2020-06-05,0.24337,0.24750,0.23970,0.24023,7754761,USD
...,...,...,...,...,...,...,...
580,2022-01-02,3.76567,3.85116,3.71833,3.83067,242553,USD
581,2022-01-03,3.83070,3.84950,3.68248,3.76424,338507,USD
582,2022-01-04,3.76462,3.88907,3.71524,3.78952,415813,USD
583,2022-01-05,3.78447,3.84644,3.41846,3.53586,582713,USD


Chuẩn hóa dữ liệu

In [9]:
df['H-L'] = df['High'] - df['Low']
df['O-C'] = df['Open'] - df['Close']
ma_1 = 7
ma_2 = 14
ma_3 = 21
# Đường Simple Moving Average (SMA)
# là đường trung bình động đơn giản được tính bằng trung bình cộng các mức giá đóng cửa trong một khoảng thời gian giao dịch nhất định.
# Vd MA_7 = Khoảng thời gian (7 ngày về sau tính cả ngày đang tính)
df[f'SMA_{ma_1}'] = df['Open'].rolling(window=ma_1).mean()
df[f'SMA_{ma_2}'] = df['Open'].rolling(window=ma_2).mean()
df[f'SMA_{ma_3}'] = df['Open'].rolling(window=ma_3).mean()

df[f'SD_{ma_1}'] = df['Open'].rolling(window=ma_1).std()
df[f'SD_{ma_3}'] = df['Open'].rolling(window=ma_3).std()
df.dropna(inplace=True)

df.to_csv(f"{stock}_8_2_standardized.csv")
print(df)

           Date     Open     High      Low    Close   Volume Currency  \
20   2020-06-21  0.22887  0.23118  0.22738  0.22813  4102533      USD   
21   2020-06-22  0.22811  0.24639  0.22769  0.24325  7929329      USD   
22   2020-06-23  0.24324  0.24477  0.24094  0.24320  5163349      USD   
23   2020-06-24  0.24320  0.24923  0.23156  0.23475  9127950      USD   
24   2020-06-25  0.23475  0.23529  0.22799  0.23256  6475813      USD   
..          ...      ...      ...      ...      ...      ...      ...   
580  2022-01-02  3.76567  3.85116  3.71833  3.83067   242553      USD   
581  2022-01-03  3.83070  3.84950  3.68248  3.76424   338507      USD   
582  2022-01-04  3.76462  3.88907  3.71524  3.78952   415813      USD   
583  2022-01-05  3.78447  3.84644  3.41846  3.53586   582713      USD   
584  2022-01-06  3.53615  3.55019  3.30614  3.40777   701888      USD   

         H-L      O-C     SMA_7    SMA_14    SMA_21      SD_7     SD_21  
20   0.00380  0.00074  0.231636  0.236451  0.2380

Scale Miền giá trị lại

In [10]:
# Process Data
pre_day = 30
scala_x = MinMaxScaler()
scala_y = MinMaxScaler()
cols_x = ['H-L', 'O-C', f'SMA_{ma_1}', f'SMA_{ma_2}', f'SMA_{ma_3}', f'SD_{ma_1}', f'SD_{ma_3}']
cols_y = ['Open']
scaled_data_x = scala_x.fit_transform(df[cols_x].values.reshape(-1, len(cols_x)))
scaled_data_y = scala_y.fit_transform(df[cols_y].values.reshape(-1, len(cols_y)))
# scaled_data_x
scaled_data_y

array([[1.70035773e-03],
       [1.53468185e-03],
       [4.83293985e-03],
       [4.82422007e-03],
       [2.98216586e-03],
       [2.50693768e-03],
       [1.88565312e-03],
       [0.00000000e+00],
       [8.76338214e-04],
       [1.48454309e-03],
       [9.89695396e-04],
       [2.15596640e-03],
       [1.16627101e-03],
       [8.76338214e-04],
       [1.74831654e-03],
       [1.49980271e-03],
       [4.46670896e-03],
       [3.96532142e-03],
       [5.63733986e-03],
       [4.56480652e-03],
       [4.39041085e-03],
       [3.95224175e-03],
       [4.74574202e-03],
       [4.07213877e-03],
       [4.23345476e-03],
       [3.82580489e-03],
       [2.74237182e-03],
       [2.55053659e-03],
       [3.19362061e-03],
       [3.91518267e-03],
       [3.27645855e-03],
       [5.33432739e-03],
       [9.20373120e-03],
       [1.18283859e-02],
       [1.27003643e-02],
       [1.83660434e-02],
       [1.96304120e-02],
       [2.19629540e-02],
       [2.09013204e-02],
       [2.10844358e-02],


In [11]:
x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])
x_total=np.array(x_total)
y_total=np.array(y_total)

Tách tập dữ liệu thành train và test

In [12]:
# Build Model
model = Sequential()

model.add(GRU(units=60, return_sequences=True, input_shape=(x_total.shape[1], x_total.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60))
model.add(Dropout(0.2))
model.add(Dense(units=len(cols_y)))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_total, y_total, epochs=60, steps_per_epoch=40, use_multiprocessing=True)
model.save(f"{stock}82.h5")
print("Done Training Model")

Epoch 1/60
40/40 [==============================] - 10s 33ms/step - loss: 0.0200
Epoch 2/60
40/40 [==============================] - 1s 33ms/step - loss: 0.0094
Epoch 3/60
40/40 [==============================] - 1s 34ms/step - loss: 0.0095
Epoch 4/60
40/40 [==============================] - 1s 34ms/step - loss: 0.0093
Epoch 5/60
40/40 [==============================] - 1s 34ms/step - loss: 0.0088
Epoch 6/60
40/40 [==============================] - 1s 34ms/step - loss: 0.0072
Epoch 7/60
40/40 [==============================] - 1s 33ms/step - loss: 0.0053
Epoch 8/60
40/40 [==============================] - 1s 33ms/step - loss: 0.0056
Epoch 9/60
40/40 [==============================] - 1s 33ms/step - loss: 0.0048
Epoch 10/60
40/40 [==============================] - 1s 33ms/step - loss: 0.0053
Epoch 11/60
40/40 [==============================] - 1s 34ms/step - loss: 0.0046
Epoch 12/60
40/40 [==============================] - 1s 34ms/step - loss: 0.0038
Epoch 13/60
40/40 [=================